# Non-Directional Price Spike Strategy

This notebook explores a mean reversion strategy that trades on price spikes regardless of direction, only considering whether the price is above or below the fair price.

In [ ]:
import sys
import os

# Import our backtester package
sys.path.append(os.path.abspath('../../'))
from backtester import get_price_data, get_vwap
print("Using backtester package")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Try to import seaborn, but don't fail if it's not available
try:
    import seaborn as sns
    print(f"Seaborn version: {sns.__version__}")
    sns.set(style="whitegrid")
    HAS_SEABORN = True
except ImportError:
    print("Seaborn not available, using matplotlib instead")
    HAS_SEABORN = False

## 1. Load Data

First, let's load the Squid_Ink price data and limit it to the first 20,000 timestamps (in-sample data).

In [ ]:
# Load data directly using backtester package
print("Loading price data...")
prices = get_price_data('SQUID_INK', 1)
print(f"Loaded {len(prices)} price data points")

# Limit to first 20,000 timestamps (in-sample data)
in_sample_prices = prices.iloc[:20000]
print(f"Limited to {len(in_sample_prices)} in-sample data points")

# Get VWAP
print("Getting VWAP for SQUID_INK...")
squid_vwap = in_sample_prices['vwap']
print(f"Got VWAP with {len(squid_vwap)} data points")
print(f"VWAP range: {squid_vwap.min()} to {squid_vwap.max()}")

# Calculate log prices and log returns
log_prices = np.log(squid_vwap)
log_returns = log_prices.diff().dropna()
print(f"Calculated log returns with {len(log_returns)} data points")

# Calculate regular returns for strategy evaluation
returns = squid_vwap.pct_change().dropna()
print(f"Calculated returns with {len(returns)} data points")

## 2. Calculate Price Spikes

Let's calculate price spikes as the absolute value of log returns divided by rolling standard deviation.

In [ ]:
def calculate_price_spikes(log_returns, window=20, absolute=False):
    """
    Calculate price spikes as log returns divided by rolling standard deviation.
    
    Parameters:
        log_returns (pd.Series): Series of log returns
        window (int): Window size for rolling standard deviation
        absolute (bool): Whether to take the absolute value of the spikes
        
    Returns:
        pd.Series: Price spikes (z-scores)
    """
    # Calculate rolling standard deviation
    rolling_std = log_returns.rolling(window=window).std()
    
    # Calculate price spikes (z-scores)
    price_spikes = log_returns / rolling_std
    
    # Take absolute value if requested
    if absolute:
        price_spikes = price_spikes.abs()
    
    return price_spikes

# Calculate directional price spikes
window = 20  # Use a fixed window size
directional_spikes = calculate_price_spikes(log_returns, window, absolute=False)

# Calculate absolute price spikes
absolute_spikes = calculate_price_spikes(log_returns, window, absolute=True)

# Display the first few rows
pd.DataFrame({'Directional Spikes': directional_spikes, 'Absolute Spikes': absolute_spikes}).head(10)

## 3. Define Non-Directional Price Spike Strategy

Let's define a strategy that trades on price spikes regardless of direction, only considering whether the price is above or below the fair price.

In [ ]:
def nondirectional_spike_strategy(prices, spikes, fair_price, threshold=2.0, holding_period=10):
    """
    Implement a non-directional price spike-based mean reversion strategy.
    
    Parameters:
        prices (pd.Series): Series of prices
        spikes (pd.Series): Series of absolute price spikes (z-scores)
        fair_price (float): Fair price to revert to
        threshold (float): Threshold for price spikes
        holding_period (int): Number of periods to hold the position
        
    Returns:
        pd.Series: Portfolio positions (1 for long, -1 for short, 0 for no position)
    """
    # Initialize positions
    positions = pd.Series(0, index=prices.index)
    
    # Get valid indices where spikes is not NaN
    valid_indices = spikes.dropna().index
    
    # Set positions based on price spikes and fair price
    for time in valid_indices:
        # Get the current price and spike value
        current_price = prices.loc[time]
        current_spike = spikes.loc[time]
        
        # Get the index position
        idx = prices.index.get_loc(time)
        
        # Only trade when there's a significant price spike (regardless of direction)
        if current_spike > threshold:
            # Short above fair price
            if current_price > fair_price:
                # Set short position for holding period
                end_idx = min(idx + holding_period + 1, len(positions))
                positions.iloc[idx+1:end_idx] = -1
            # Buy below fair price
            elif current_price < fair_price:
                # Set long position for holding period
                end_idx = min(idx + holding_period + 1, len(positions))
                positions.iloc[idx+1:end_idx] = 1
    
    return positions

## 4. Test Threshold Parameters

Let's test different threshold parameters for the non-directional price spike strategy, focusing on total returns as the primary metric.

In [ ]:
# Define the fair price
FAIR_PRICE = 2000

# Define threshold parameters to test
thresholds = [1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0]
holding_period = 10  # Use a fixed holding period for threshold testing

# Initialize results dictionary
threshold_results = {}

# Test different threshold parameters
for threshold in thresholds:
    # Get positions
    positions = nondirectional_spike_strategy(squid_vwap, absolute_spikes, FAIR_PRICE, threshold, holding_period)
    
    # Calculate strategy returns
    strategy_returns = positions.shift(1) * returns
    strategy_returns = strategy_returns.dropna()
    
    # Calculate performance metrics
    total_return = strategy_returns.sum()
    sharpe_ratio = strategy_returns.mean() / strategy_returns.std() * np.sqrt(252)  # Annualized
    win_rate = (strategy_returns > 0).mean()
    
    # Count the number of trades
    num_trades = (positions.diff() != 0).sum()
    
    # Store results
    threshold_results[threshold] = {
        'Threshold': threshold,
        'Total Return': total_return,
        'Sharpe Ratio': sharpe_ratio,
        'Win Rate': win_rate,
        'Number of Trades': num_trades
    }

# Convert results to DataFrame
threshold_df = pd.DataFrame(threshold_results).T

# Sort by Total Return
threshold_df = threshold_df.sort_values('Total Return', ascending=False)

# Display results
threshold_df